In [1]:
# -*- coding: utf-8 -*-

"""
This ILS (iterated local search) algorithm tries to minimize the cost of the links in a telecommunication network given specific constraints.
"""

import pandas as pd
import numpy as np
import copy


# Input data preperation
# The excel file must be in the same folder, othwise the path to the excel file must be specified
filename = "InputDataTelecomLargeInstance.xlsx"
# filename = "InputDataTelecomSmallInstance.xlsx"
C = pd.read_excel(filename, sheet_name="C", header=None)
M = pd.read_excel(filename, sheet_name="M", header=None)
N = pd.read_excel(filename, sheet_name="N", header=None)
h = pd.read_excel(filename, sheet_name="CustToTargetAllocCost(hij)", header=None)
c = pd.read_excel(filename, sheet_name="TargetToSteinerAllocCost(cjk)", header=None)
g = pd.read_excel(filename, sheet_name="SteinerToSteinerConnctCost(gkm)", header=None)
f = pd.read_excel(filename, sheet_name="SteinerFixedCost(fk)", header=None)
U = pd.read_excel(filename, sheet_name="TargetCapicity(Uj)", header=None)
V = pd.read_excel(filename, sheet_name="SteinerCapacity(Vk)", header=None)
alpha = pd.read_excel(filename, sheet_name="alpha", header=None)

C = C[0][0] # number of customers
M = M[0][0] # number of end offices
N = N[0][0] # number of digital offices
alpha = alpha[0][0] # proportion of customers that must be connected
h = np.array(h) # customer-office link costs
c = np.array(c) # office-hub link costs
g = np.array(g) # hub-hub link costs
f = np.array(f) # hub selection costs
U = np.array(U) # maximum customers offices can have
V = np.array(V, dtype=np.int32) # maximum customers hubs can have

f = f.flatten()
U = U.flatten()
V = V.flatten()

set_C = np.arange(C) # set of customers
set_M = np.arange(M) # set of end offices
set_N = np.arange(N) # set of digital hubs
# Values in the sets start at 0


"""
The OneRandomSolution class is called to instantiate a random solution
When an instantiated solution is printed you get:
- customer_office_links: if customer i is linked to office j, the i-th value is equal to j
                         if customer is not linked the value is -1
- nb_customers_per_office: if office i has k customers, the i-th value is equal to k
- office_hub_links: if office i is linked to hub j, the i-th value is equal to j
                    if office is not linked the value is -1
- nb_customers_per_hub: if hub i has k customer, the i-th value is equal to k
- digital_hubs_sequence: order in which digital hubs are linked in the ring
                        if hub i is not selected it is not in the list
- Objective value: total cost for this solution
"""
class OneRandomSolution:
    
    def __init__(self):
        self.initialise_random_sets()
        self.find_solution()
        self.total_cost = self.get_cost()
        
    def __repr__(self):
        text = "Solution :"
        text += "\ncustomer_office_links : " + str(self.customer_office_links)
        text += "\nnb_customers_per_office : " + str(self.nb_customers_per_office)
        text += "\noffice_hub_links : " + str(self.office_hub_links)
        text += "\nnb_customers_per_hub : " + str(self.nb_customers_per_hub)
        text += "\ndigital_hubs_sequence : " + str(self.digital_hubs_sequence)
        text += "\nObjective value : " + str(self.total_cost)
        return text
        
  
    def initialise_random_sets(self):
        self.mixed_set_customers = np.random.permutation(set_C)
        self.mixed_set_offices = np.random.permutation(set_M)
        self.mixed_set_hubs = np.random.permutation(set_N)
    
    
    def find_solution(self):
        mixed_set_offices = self.mixed_set_offices
        mixed_set_hubs = self.mixed_set_hubs
        
        # First we link customers to end offices
        customer_office_links = np.full(C, -1, dtype=np.int32)
        nb_customers_per_office = np.zeros(M, dtype=np.int32)
        min_allocated_custumers = int(alpha * C) + 1

        # idx_c refers to the index of the mixed set of customers
        current_idx_c = 0
        for office in mixed_set_offices:
            next_idx_c = current_idx_c + U[office]
            if next_idx_c <= min_allocated_custumers:
                customer_office_links[current_idx_c : next_idx_c] = np.full(U[office], office)
                current_idx_c = next_idx_c
                nb_customers_per_office[office] += U[office]
            else:
                last_index = min_allocated_custumers
                customer_office_links[current_idx_c : last_index] = np.full(last_index - current_idx_c, office)
                nb_customers_per_office[office] += last_index - current_idx_c
                break
        
        # Now we link end offices to digital hubs
        office_hub_links = np.full(M, -1, dtype=np.int32)
        nb_customers_per_hub = np.zeros(N, dtype=np.int32)

        # idx_o refers to the index of the mixed set of end-offices
        current_idx_o = 0
        next_idx_o = 0
        for hub in mixed_set_hubs:
            while nb_customers_per_hub[hub] + nb_customers_per_office[next_idx_o] <= V[hub]:
                nb_customers_per_hub[hub] += nb_customers_per_office[next_idx_o]
                next_idx_o += 1
                if next_idx_o == M:
                    break
            office_hub_links[current_idx_o : next_idx_o] = hub
            current_idx_o = next_idx_o
            if current_idx_o == M:
                break
                
        # Finally we link digital hubs together so that they form a ring
        digital_hubs_sequence = list(np.where(nb_customers_per_hub != 0)[0])
        not_in_sequence = list(np.where(nb_customers_per_hub == 0)[0])
        
        # To make sure at leaste 3 hubs are activated:
        while len(digital_hubs_sequence) < 3:
            if not_in_sequence == []:
                raise Warning("There are less than 3 hubs in input data -> No solution")
            digital_hubs_sequence.append(not_in_sequence.pop())
        
        # We create the attributes of the solution
        self.customer_office_links = customer_office_links[self.mixed_set_customers]
        self.nb_customers_per_office = nb_customers_per_office
        self.office_hub_links = office_hub_links
        self.nb_customers_per_hub = nb_customers_per_hub
        self.digital_hubs_sequence = digital_hubs_sequence
     
        
    def get_cost(self):
        # We calculate and return the total cost = value of the objective function
        
        cost_hub_alloc = 0
        for i in self.digital_hubs_sequence:
            cost_hub_alloc += f[i]

        cost_hub_hub_links = g[self.digital_hubs_sequence[-1], self.digital_hubs_sequence[0]]
        nb_alloc_hubs = len(self.digital_hubs_sequence)

        for i in range(1, nb_alloc_hubs):
            cost_hub_hub_links += g[self.digital_hubs_sequence[i], self.digital_hubs_sequence[i-1]]
        cost_hub_hub_links


        cost_office_hub_links = 0
        for office, hub in enumerate(self.office_hub_links):
            cost_office_hub_links += c[office][hub]


        cost_customer_office_link = 0
        for customer, office in enumerate(self.customer_office_links):
            cost_customer_office_link += h[customer][office]

        total_cost = cost_hub_alloc + cost_hub_hub_links + cost_office_hub_links + cost_customer_office_link
        return total_cost

    def recalculate_cost(self):
        self.total_cost = self.get_cost()
        return self.total_cost


def global_search(n):
    # Generates a list of n random solutions
    list_solutions = []
    for i in range(n):
        one_solution = OneRandomSolution()
        list_solutions.append(one_solution)
    return list_solutions


def find_best_sol(list_solution):
    # Returns the solution with lowest objective value
    best_sol = list_solution[0]
    min_cost = best_sol.total_cost
    for sol in list_solution[1:]:
        if sol.total_cost < min_cost:
            min_cost = sol.total_cost
            best_sol = sol
    return best_sol
        

def swap_two_elements(old_set):
    # Swap the position of two elements in a list randomly
    elem_a = np.random.choice(old_set)
    elem_b = np.random.choice(np.delete(old_set, elem_a))
    new_set = np.copy(old_set)
    new_set[elem_a] = old_set[elem_b]
    new_set[elem_b] = old_set[elem_a]
    return new_set
    

    
def try_swap_two_customers(old_solution):
    # Generates a new solution by swapping randomly two customers
    # Returns the new solution, if it has a lower or equal objective value
    old_obj_value = old_solution.total_cost
    old_set_C = old_solution.customer_office_links
    new_set_C = swap_two_elements(old_set_C)
    new_solution = copy.copy(old_solution)
    new_solution.customer_office_links = new_set_C
    new_obj_value = new_solution.recalculate_cost()
    if new_obj_value <= old_obj_value:
        return new_solution
    return old_solution


def local_search(start_solution, n_iter):
    current_solution = start_solution
    for i in range(n_iter):
        current_solution = try_swap_two_customers(current_solution)
        # The objective value of the current solution can only decrease or stay equal
    return current_solution



def main():
    n_global = 800
    n_local = 400
    # We generate a first list of random solutions
    first_list_solutions = global_search(n_global)
    # The second list will store those solutions after a local search
    second_list_solution = []
    for i, solution in enumerate(first_list_solutions):
        new_sol = local_search(solution, n_local)
        second_list_solution.append(new_sol)
    # We return the best solution of the second list
    best_solution = find_best_sol(second_list_solution)
    return best_solution



main()

Solution :
customer_office_links : [11 14  1 10 14 11 11 -1  1 14 -1 14  1 10 10 10 10 11 14 14 10 10 10 11
 11  1 14 10 11 11 14 11 -1 10 14  1 11 11 14 11  1  1 -1  1 11 14 11  1
  1  1]
nb_customers_per_office : [ 0 11  0  0  0  0  0  0  0  0 10 14  0  0 11]
office_hub_links : [ 1  4  4  4  4  4  4  4  4  4  3 -1 -1 -1 -1]
nb_customers_per_hub : [ 0  0  0 10 11  0  0  0  0  0]
digital_hubs_sequence : [3, 4, 9]
Objective value : 8817